# Import SOCAT data
This notebook reads the contents of the main SOCAT synthesis file and created a PostGIS database ready for analysis.

## Prerequisites
This notebook assumes that you have created a PostgreSQL database with the PostGIS extension installed:
- `CREATE DATABASE socat_kpi;`
- `CREATE EXTENSION postgis;`

## Setup
Imports, constants etc.

In [26]:
import psycopg2
from tqdm.notebook import tqdm
from datetime import datetime
import re
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

AD = ('https://socat.info/socat_files/v2025/SOCATv2025.tsv.zip', 'SOCATv2025.tsv', 'A-D Data')
E = ('https://socat.info/socat_files/v2025/SOCATv2025_FlagE.tsv.zip', 'SOCATv2025_FlagE.tsv', 'E Data')

DB_HOST = 'localhost'
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_NAME = 'socat_kpi'

## Download SOCAT data
If the SOCAT data files are not present, download them.

In [27]:
def download_file(file_info):
    url = file_info[0]
    file = file_info[1]
    description = file_info[2]

    
    if os.path.exists(file):
        print(f'{description} already downloaded.')
    else:
        print(f'Downloading {description}...')
        resp = urlopen(url)
        myzip = ZipFile(BytesIO(resp.read()))
        print('Extracting data...')
        myzip.extract(file)
        myzip.close()
        print('Done')

download_file(AD)
print()
download_file(E)


A-D Data already downloaded.

E Data already downloaded.


## Connect to database
Connect to the database and delete any existing tables.

In [38]:
conn = psycopg2.connect(database = DB_NAME, 
                        user = DB_USER, 
                        host= DB_HOST,
                        password = DB_PASSWORD)

cur = conn.cursor()

## Kill the old table

In [39]:
cur.execute("DROP TABLE IF EXISTS socat")
conn.commit()

## Create SOCAT data table
We will create the following fields:
- Platform Code
- EXPO Code
- Timestamp
- Year†
- Month†
- Day†
- Month_Day†‡
- Lat/Lon
- fCO₂ value
- fCO₂ flag

† Useful for generating stats

‡ Combined month and day

In [40]:
cur.execute("""CREATE TABLE socat(
id bigserial primary key,
platform_code varchar(5),
country varchar(2),
expocode varchar(20),
time timestamp,
year int,
month int,
day int,
month_day int,
position geometry(Point, 4326),
fco2 float,
fco2_flag integer,
socat_flag varchar(1)
);""")
conn.commit()

## Locate the data in the SOCAT file
We skip the header. There are 3 lines starting with `Expocode` before the data starts. (The last is the column header line, but we don't need it.)

In [41]:
def load_data(file_info):

    file = file_info[1]
    description = file_info[2]
    
    print(f'Loading {description}')
    
    with open(file, 'rb') as f:
        line_count = sum(1 for _ in f)
    
    f = open(file)
    
    with tqdm(total=line_count) as progress:
        expocode_count = 0
        while expocode_count < 3:
            line = f.readline()
            if line.startswith('Expocode'):
                expocode_count += 1
            progress.update()
            
        line = f.readline()
        record_count = 0
        progress.update()
        while line != '':
            fields = line.split('\t')
            expocode = fields[0]
            if '-' in expocode:
                platform_code = re.search('(.*)\\d{8}-\\d$', expocode)[1]
            else:
                platform_code = re.search('(.*)\\d{8}$', expocode)[1]
    
            country = platform_code[0:2]
    
            seconds = int(fields[9][0:2])
            if seconds > 59:
                seconds = 59
            timestamp = datetime(int(fields[4]), int(fields[5]), int(fields[6]), int(fields[7]), int(fields[8]), seconds)
            month_day = int(fields[5]) * 100 + int(fields[6])
            fco2 = float(fields[29])
            fco2_flag = int(fields[31])
            socat_flag = fields[3]
    
            lon = float(fields[10])
            if lon > 180:
                lon = (360 - lon) * -1
            
            cur.execute(f"""INSERT INTO socat (platform_code, country, expocode, time, year, month, day, month_day, position, fco2, fco2_flag, socat_flag) VALUES 
            ('{platform_code}', '{country}', '{expocode}', '{timestamp}', {int(fields[4])}, {int(fields[5])}, {int(fields[6])}, {month_day}, ST_GeomFromText('POINT({lon} {float(fields[11])})', 4326), '{fco2}', '{fco2_flag}', '{socat_flag}')""")
    
            record_count += 1
            #if record_count % 1000000 == 0:
            #    conn.commit()
    
            progress.update()
            line = f.readline()

    conn.commit()        
    f.close()

load_data(AD)
load_data(E)

Loading A-D Data


  0%|          | 0/41387663 [00:00<?, ?it/s]

Loading E Data


  0%|          | 0/8210291 [00:00<?, ?it/s]

## Create index
Create a spatial index to speed up searches

In [42]:
print('Creating indexes (approx. 5 min)...')
print('Points')
cur.execute('CREATE INDEX socat_points_idx ON socat USING GIST(position)')
print('Year')
cur.execute('CREATE INDEX idx_socat_year ON socat(year)')
print('Month')
cur.execute('CREATE INDEX idx_socat_month ON socat(month)')
print('Month/Day combined')
cur.execute('CREATE INDEX idx_socat_monthday ON socat(month_day)')
print('Platform code')
cur.execute('CREATE INDEX idx_socat_platformcode ON socat(platform_code)')
print('Country')
cur.execute('CREATE INDEX idx_socat_country ON socat(country)')
print('SOCAT Flag')
cur.execute('CREATE INDEX idx_socat_socatflag ON socat(socat_flag)')
print('Done')
conn.commit()

Creating indexes (approx. 5 min)...
Points
Year
Month
Month/Day combined
Platform code
Country
SOcAT Flag
Done


## Close everything down

In [44]:
cur.close()
conn.close()